In [1]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [2]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [3]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

# Read and Write Functions

In [13]:
# Defining Function for writing in files (we will use to convert lists to .txt files)
def write_files(file_path, lst):
    with open(file_path, 'w', encoding='utf-8') as file:
        for item in lst:
            file.write("%s\n" % item)

# Defining Function which will append eachsentence as a string in the list of string 
hindi_sentences = []
english_sentences = []
gujarati_sentences = []

    # Reading each file and extracting the sentences in form of lists
with open('/kaggle/input/datasetsg/ground_truth_hin.txt', 'r', encoding='utf-8') as file:
    hindi_sentences = file.readlines()
with open('/kaggle/input/datasetsg/ground_truth_eng.txt', 'r', encoding='utf-8') as file:
    english_sentences = file.readlines()           
with open('/kaggle/input/datasetsg/ground_truth_guj.txt', 'r', encoding='utf-8') as file:
    gujrati_sentences = file.readlines()
# file_paths = ("/kaggle/input/datasetsg/ground_truth_hin.txt", "/kaggle/input/datasetsg/ground_truth_eng.txt", "/kaggle/input/datasetsg/ground_truth_guj.txt")
# hindi_sentences_1k, english_sentences_1k, gujarati_sentences_1k = read_files(file_paths)


In [14]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [15]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [ ]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(english_sentences, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)
write_files("english_to_hindi_indic.txt", hi_translations)


# delete the models to free the GPU memory
# del en_indic_tokenizer, en_indic_model

In [ ]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "indic-en", quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_translations = batch_translate(hindi_sentences, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)
print(en_translations)
write_files("hindi_to_english_indic.txt", en_translations)

In [17]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "hin_Deva", "guj_Gujr"
hi_to_guj_translations = batch_translate(hindi_sentences, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)
write_files("hindi_to_gujrati_indic.txt", hi_to_guj_translations)

In [18]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

src_lang, tgt_lang = "guj_Gujr", "hin_Deva"
gu_translations = batch_translate(hindi_sentences, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)
write_files("gujrati_to_hindi_indic.txt", gu_translations)